# AMLS Estimator for Marge vs Home

In [19]:
import azureml.core
from azureml.core import Workspace, Datastore, Experiment
from azureml.core.model import Model
from azureml.core.compute import ComputeTarget, AmlCompute, DataFactoryCompute
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.core.runconfig import CondaDependencies, RunConfiguration
from azureml.data.data_reference import DataReference
from azureml.pipeline.core.graph import PipelineParameter
from azureml.pipeline.steps import DataTransferStep, PythonScriptStep, EstimatorStep
from azureml.data.datapath import DataPath, DataPathComputeBinding
from azureml.widgets import RunDetails
from azureml.train.dnn import TensorFlow
import os

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.69


## Connect to Workspace

In [20]:
ws = Workspace.from_config()

ws.get_details()

{'id': '/subscriptions/431dbae5-40ca-438a-8daf-77d7d5580b41/resourceGroups/AMLS-Demo-RG/providers/Microsoft.MachineLearningServices/workspaces/AMLS-Workspace',
 'name': 'AMLS-Workspace',
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'workspaceid': '7762f3db-1959-4f1c-aac2-b8d3e035f52b',
 'description': '',
 'friendlyName': '',
 'creationTime': '2019-09-10T08:07:46.7167284+00:00',
 'containerRegistry': '/subscriptions/431dbae5-40ca-438a-8daf-77d7d5580b41/resourceGroups/AMLS-Demo-RG/providers/Microsoft.ContainerRegistry/registries/amlsworkspac2bb6e513',
 'keyVault': '/subscriptions/431dbae5-40ca-438a-8daf-77d7d5580b41/resourcegroups/amls-demo-rg/providers/microsoft.keyvault/vaults/amlsworkspace3533516247',
 'applicationInsights': '/subscriptions/431dbae5-40ca-438a-8daf-77d7d5580b41/resourcegroups/amls-demo-rg/providers/microsoft.insights/components/amlsworkspace6001352158',
 'identityPrincipalId': '89da5ddd-ca21-4e89-a603-e4373100ffef',

## Connect to Compute

In [21]:
clusterName = "NV6AICluster"
computeCluster = ComputeTarget(workspace=ws, name=clusterName)


# Get some details from your cluster
status = computeCluster.get_status()
print("Current node count: ",status.current_node_count)
print("VM Size: ",status.vm_size)
print("Name:",computeCluster.name)

Current node count:  1
VM Size:  STANDARD_NV6
Name: NV6AICluster


## Connect to the datastore

In [22]:
source_ds = Datastore.get(ws, 'SimpsonDataStore')

## Create the Experiment

In [23]:
ExperimentName = "Simpsons-AI"

exp = Experiment(workspace=ws, name=ExperimentName)
print(exp.name)

Simpsons-AI


## Creating the Estimator

In [24]:
script_folder = "./trainingscripts"
script_params = {
    "--data-folder": source_ds.as_mount(),
    "--pic-size":64,
    "--batch-size":32,
    "--epochs":10
}

estimator = TensorFlow (
     source_directory = script_folder,
     script_params = script_params,
     compute_target = computeCluster,
     entry_script = "train.py", 
     use_gpu = True,
     use_docker = True,
     conda_packages=["keras==2.2.2","opencv==3.4.2","scikit-learn"],
     framework_version="1.10"
)

In [25]:
run = exp.submit(estimator)

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Register the model

In [8]:
model_file_h5 = "outputs/simple_model.h5"
model_file_json = "outputs/simple_model.json"

os.makedirs("outputs", exist_ok=True)
run.download_file(name=model_file_h5, output_file_path="outputs/simple_model.h5")
run.download_file(name=model_file_json, output_file_path="outputs/simple_model.json")

model = Model.register(ws, 
               model_name="SimpsonsAI-NDC", 
               model_path="outputs", 
               tags = {"conference": "NDC SYDNEY"}, 
               description="Simpson classification model created at NDC Sydney")

WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', timeout())': /azureml/LocalUpload/191017T070523-aa6ccdc1/SimpsonsAI-NDC.tar.gz?sv=2018-11-09&sr=b&sig=YP2x3R5l6Jg16GH3WZ8MxuMafnBW3zIDS4Ajqzdx%2Bfg%3D&st=2019-10-17T04%3A55%3A23Z&se=2019-10-18T05%3A05%3A23Z&sp=rcw&comp=block&blockid=TURBd01EQXdNREF3TURBd01EQXdNREF3TURBd01EQXdNRFF4T1RRek1EUSUzRA%3D%3D&timeout=30


Registering model SimpsonsAI-NDC


## Download the model

In [9]:
model = Model(ws, 'SimpsonsAI-NDC')

print("Name",model.name)
print("Version",model.version)

output_path = "."
#os.makedirs(output_path, exist_ok=True)

result = model.download(target_dir=output_path, exist_ok=True)

#print(result)

Name SimpsonsAI-NDC
Version 2


# Testing the model localy

## Load the model

In [17]:
import os
import shutil
import numpy as np
import cv2
import random
import keras
from keras.models import model_from_json

### Settings
json_file_path = "outputs/simple_model.json"
model_file_path = "outputs/simple_model.h5"
test_images_directory = "testset-marge-vs-homer"
pic_size = 64

### characters
map_characters = {0:"marge_simpson", 1: "homer_simpson" }

### Load the model
json_file = open(json_file_path, "r") 
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(model_file_path)  

### Prediction method => image preprocessing + prediction
def predict_image(img):
    image = cv2.imread(img)
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pic = cv2.resize(image, (pic_size,pic_size))
    data = pic.reshape(1, pic_size, pic_size,3)
    predicted_labels = loaded_model.predict(data)[0]
    return predicted_labels, img

print("Loaded")

Using TensorFlow backend.


Loaded


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid

F = plt.figure(1, (20,20))
grid = AxesGrid(F, 111, nrows_ncols=(3, 3), axes_pad=0, label_mode="1")

testimages = os.listdir(test_images_directory)
random.shuffle(testimages)

i = 0
for img_name in testimages[0:9]:
    a, img = predict_image(os.path.join(test_images_directory,img_name))

    text = sorted(["{:s} : {:.1f}%".format(map_characters[k].split("_")[0].title(), 100*v) for k,v in enumerate(a)], 
       key=lambda x:float(x.split(":")[1].split("%")[0]), reverse=True)[:3]
    img = cv2.resize(img, (352, 352))
    cv2.rectangle(img, (0,260),(215,352),(255,255,255), -1)
    font = cv2.FONT_HERSHEY_SIMPLEX

    for k, t in enumerate(text):
        cv2.putText(img, t,(10, 300+k*25), font, 0.75,(0,0,0),2,cv2.LINE_AA)
    grid[i].imshow(img)
    i = i +1